In [1]:
# Import necessary libraries
import sqlite3
import pandas as pd

# Connect to a SQLite database (or create one)
conn = sqlite3.connect('./data/bookstore.db')
cursor = conn.cursor()

#output
#4 extra tables - at the end of normalization

### Exercise
- Normalize the table appropriately and create logical foreign keys
- Extract all book genres and list books belonging to them
- Extract all publishers and list all of their published books
- Assign an extra e-mail under the domain ###@alphapubs.com for each author publishing with 'Alpha Publications'. Adjust the database schema appropriately to achieve this.

In [2]:
# cursor.description - contains a tuple of tuples with names of the columns in a table
# columns = next(zip(*cursor.description)) #1st element of the tuple will contains all first elements from all tuples
#next at the beginning of the tuple will take the first tuple inside the tuple

In [3]:
# to get table name from the database

#cursor.execute("SELECT * FROM bookstore.INFORMATION_SCHEMA.TABLES") #sqlite3 does not have this query

cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")

table = cursor.fetchone()
table_name = table[0]


In [4]:
cursor.execute("SELECT * FROM unnormalized_books")
row = cursor.fetchone()
print(row)

(1, 'The Great Escape', 'John Doe', 'john@example.com', 'Adventure', 10.99, 'Alpha Publications', 'contact@alpha.com')


# creating table authors

In [5]:
cursor.execute('DROP TABLE authors;')
cursor.execute('''
CREATE TABLE authors(
    author_id INTEGER PRIMARY KEY,
    author_name TEXT NOT NULL,
    email TEXT NOT NULL
)
''')

In [6]:
cursor.execute("INSERT INTO authors (author_name, email) SELECT DISTINCT author_name, author_email  FROM unnormalized_books")

In [7]:
cursor.execute("SELECT * FROM authors")
rows = cursor.fetchall()

for row in rows:
    print(row)

(1, 'John Doe', 'john@example.com')
(2, 'Jane Smith', 'jane@example.com')
(3, 'Sarah Connor', 'sarah@example.com')
(4, 'Mike Tyson', 'mike@example.com')
(5, 'Lucy Loo', 'lucy@example.com')
(6, 'Brian Bryson', 'brian@example.com')
(7, 'Anna Smith', 'anna@example.com')
(8, 'Tim Track', 'tim@example.com')
(9, 'Liam Neeson', 'liam@example.com')
(10, 'Emily Em', 'emily@example.com')
(11, 'Oscar Orange', 'oscar@example.com')
(12, 'Jenny June', 'jenny@example.com')
(13, 'Tom Thumb', 'tom@example.com')
(14, 'Penny Pen', 'penny@example.com')
(15, 'Ronny Ron', 'ronny@example.com')
(16, 'Samantha Sam', 'samantha@example.com')
(17, 'Danny Day', 'danny@example.com')
(18, 'Wendy Wind', 'wendy@example.com')
(19, 'Vince Vortex', 'vince@example.com')


In [8]:
cursor.execute('''
SELECT unnormalized_books.author_name
FROM unnormalized_books
JOIN authors ON unnormalized_books.author_name = authors.author_name
''')
cursor.fetchall()


[('John Doe',),
 ('Jane Smith',),
 ('John Doe',),
 ('Sarah Connor',),
 ('Mike Tyson',),
 ('Lucy Loo',),
 ('Brian Bryson',),
 ('Anna Smith',),
 ('Tim Track',),
 ('Liam Neeson',),
 ('Emily Em',),
 ('Oscar Orange',),
 ('Jenny June',),
 ('Tom Thumb',),
 ('Penny Pen',),
 ('Ronny Ron',),
 ('Samantha Sam',),
 ('Danny Day',),
 ('Wendy Wind',),
 ('Vince Vortex',)]

# creating table publishers

In [9]:
cursor.execute('DROP TABLE publishers;')

In [10]:

cursor.execute('''
CREATE TABLE publishers(
    publisher_id INTEGER PRIMARY KEY,
    publisher_name TEXT NOT NULL,
    contact_email TEXT NOT NULL
)
''')

In [11]:
cursor.execute('''
INSERT INTO publishers(
    publisher_name, contact_email
)
SELECT DISTINCT unnormalized_books.publisher, unnormalized_books.publisher_contact
FROM unnormalized_books
''')
print(cursor.rowcount)

17


In [12]:
cursor.execute("SELECT * FROM publishers")
rows = cursor.fetchall()

for row in rows:
    print(row)

(1, 'Alpha Publications', 'contact@alpha.com')
(2, 'Beta Books', 'contact@beta.com')
(3, 'Gamma Publishing', 'contact@gamma.com')
(4, 'Delta Literature', 'contact@delta.com')
(5, 'Epsilon Editions', 'contact@epsilon.com')
(6, 'Zeta Publishing', 'contact@zeta.com')
(7, 'Theta Print', 'contact@theta.com')
(8, 'Iota Inc', 'contact@iota.com')
(9, 'Kappa Publishing', 'contact@kappa.com')
(10, 'Lambda Literature', 'contact@lambda.com')
(11, 'Mu Magazines', 'contact@mu.com')
(12, 'Nu Novels', 'contact@nu.com')
(13, 'Xi Xerox', 'contact@xi.com')
(14, 'Omicron Oasis', 'contact@omicron.com')
(15, 'Pi Press', 'contact@pi.com')
(16, 'Rho Reads', 'contact@rho.com')
(17, 'Sigma Stories', 'contact@sigma.com')


# create table books

In [13]:
cursor.execute('DROP TABLE books;')

In [14]:
cursor.execute('''
CREATE TABLE books(
    book_id INTEGER PRIMARY KEY,
    author_id INTEGER,
    publisher_id INTEGER,
    title TEXT NOT NULL,
    genre TEXT NOT NULL,
    price FLOAT,
    FOREIGN KEY (author_id) REFERENCES authors(author_id)
    FOREIGN KEY (publisher_id) REFERENCES publishers(publisher_id)
)
''')

In [15]:
cursor.execute('''
INSERT INTO books(
    author_id, publisher_id, title, genre, price
)
SELECT DISTINCT authors.author_id, publishers.publisher_id, unnormalized_books.title, unnormalized_books.genre, unnormalized_books.price
FROM unnormalized_books
JOIN authors ON unnormalized_books.author_name = authors.author_name
JOIN publishers ON unnormalized_books.publisher = publishers.publisher_name
''')


In [16]:
cursor.execute("""SELECT * FROM authors""").fetchall()

[(1, 'John Doe', 'john@example.com'),
 (2, 'Jane Smith', 'jane@example.com'),
 (3, 'Sarah Connor', 'sarah@example.com'),
 (4, 'Mike Tyson', 'mike@example.com'),
 (5, 'Lucy Loo', 'lucy@example.com'),
 (6, 'Brian Bryson', 'brian@example.com'),
 (7, 'Anna Smith', 'anna@example.com'),
 (8, 'Tim Track', 'tim@example.com'),
 (9, 'Liam Neeson', 'liam@example.com'),
 (10, 'Emily Em', 'emily@example.com'),
 (11, 'Oscar Orange', 'oscar@example.com'),
 (12, 'Jenny June', 'jenny@example.com'),
 (13, 'Tom Thumb', 'tom@example.com'),
 (14, 'Penny Pen', 'penny@example.com'),
 (15, 'Ronny Ron', 'ronny@example.com'),
 (16, 'Samantha Sam', 'samantha@example.com'),
 (17, 'Danny Day', 'danny@example.com'),
 (18, 'Wendy Wind', 'wendy@example.com'),
 (19, 'Vince Vortex', 'vince@example.com')]

In [17]:
pd.read_sql_query("""SELECT DISTINCT unnormalized_books.*, authors.*, publishers.* FROM unnormalized_books
LEFT JOIN authors ON unnormalized_books.author_name = authors.author_name
LEFT JOIN publishers ON unnormalized_books.publisher = publishers.publisher_name
                  """, conn)

,id,title,author_name,author_email,genre,price,publisher,publisher_contact,author_id,author_name,email,publisher_id,publisher_name,contact_email
0,1,The Great Escape,John Doe,john@example.com,Adventure,10.99,Alpha Publications,contact@alpha.com,1,John Doe,john@example.com,1,Alpha Publications,contact@alpha.com
1,2,Mystery Lane,Jane Smith,jane@example.com,Mystery,12.99,Beta Books,contact@beta.com,2,Jane Smith,jane@example.com,2,Beta Books,contact@beta.com
2,3,Adventure in Space,John Doe,john@example.com,Sci-fi,15.99,Alpha Publications,contact@alpha.com,1,John Doe,john@example.com,1,Alpha Publications,contact@alpha.com
3,4,Ocean Blues,Sarah Connor,sarah@example.com,Romance,7.99,Gamma Publishing,contact@gamma.com,3,Sarah Connor,sarah@example.com,3,Gamma Publishing,contact@gamma.com
4,5,Lost in Dreams,Mike Tyson,mike@example.com,Fantasy,9.99,Delta Literature,contact@delta.com,4,Mike Tyson,mike@example.com,4,Delta Literature,contact@delta.com
5,6,Sunset Stories,Lucy Loo,lucy@example.com,Drama,8.99,Epsilon Editions,contact@epsilon.com,5,Lucy Loo,lucy@example.com,5,Epsilon Editions,contact@epsilon.com
6,7,Moonlit Secrets,Brian Bryson,brian@example.com,Mystery,11.99,Zeta Publishing,contact@zeta.com,6,Brian Bryson,brian@example.com,6,Zeta Publishing,contact@zeta.com
7,8,Starstruck,Anna Smith,anna@example.com,Romance,7.99,Beta Books,contact@beta.com,7,Anna Smith,anna@example.com,2,Beta Books,contact@beta.com
8,9,Dungeon Dive,Tim Track,tim@example.com,Adventure,13.99,Alpha Publications,contact@alpha.com,8,Tim Track,tim@example.com,1,Alpha Publications,contact@alpha.com
9,10,Hopes and Dreams,Liam Neeson,liam@example.com,Drama,6.99,Theta Print,contact@theta.com,9,Liam Neeson,liam@example.com,7,Theta Print,contact@theta.com


In [18]:
cursor.execute("SELECT * FROM books")
rows = cursor.fetchall()

df = pd.DataFrame(rows)
pd.read_sql("select * from books", conn)



,book_id,author_id,publisher_id,title,genre,price
0,1,1,1,The Great Escape,Adventure,10.99
1,2,2,2,Mystery Lane,Mystery,12.99
2,3,1,1,Adventure in Space,Sci-fi,15.99
3,4,3,3,Ocean Blues,Romance,7.99
4,5,4,4,Lost in Dreams,Fantasy,9.99
5,6,5,5,Sunset Stories,Drama,8.99
6,7,6,6,Moonlit Secrets,Mystery,11.99
7,8,7,2,Starstruck,Romance,7.99
8,9,8,1,Dungeon Dive,Adventure,13.99
9,10,9,7,Hopes and Dreams,Drama,6.99


# - Extract all book genres and list books belonging to them

In [ ]:
# for key, subgroup in groupby([(1, 'a'), (2, 'b'), (2, 'c')], key=lambda item: item[0]):
#     print(key, list(subgroup))

1 [(1, 'a')]
2 [(2, 'b'), (2, 'c')]


In [20]:
from itertools import groupby

cursor.execute(''' 
SELECT genre, title
FROM books
ORDER BY genre ASC
''')
genre_title = cursor.fetchall()

for genre, (subgroup) in groupby(genre_title, key=lambda row: row[0]):
    print(f'{genre}: {", ".join([book for genre, book in subgroup])}')


Adventure: The Great Escape, Dungeon Dive, Whispering Woods
Drama: Sunset Stories, Hopes and Dreams, Deserted Destinies, Timeless Tales
Fantasy: Lost in Dreams, Enchanted Realms, Cursed Kingdoms
Horror: Hunted Houses
Mystery: Mystery Lane, Moonlit Secrets
Romance: Ocean Blues, Starstruck, Endless Embers
Sci-fi: Adventure in Space, Galactic War
Thriller: Twisted Fate, Frozen Fears


In [22]:
",".join([str(x) for x in range(10)])

'0,1,2,3,4,5,6,7,8,9'

# - Extract all publishers and list all of their published books

In [23]:
cursor.execute(''' 
SELECT books.title, publishers.publisher_name
FROM publishers
JOIN books on publishers.publisher_id = books.publisher_id
''')
cursor.fetchall()

[('The Great Escape', 'Alpha Publications'),
 ('Mystery Lane', 'Beta Books'),
 ('Adventure in Space', 'Alpha Publications'),
 ('Ocean Blues', 'Gamma Publishing'),
 ('Lost in Dreams', 'Delta Literature'),
 ('Sunset Stories', 'Epsilon Editions'),
 ('Moonlit Secrets', 'Zeta Publishing'),
 ('Starstruck', 'Beta Books'),
 ('Dungeon Dive', 'Alpha Publications'),
 ('Hopes and Dreams', 'Theta Print'),
 ('Enchanted Realms', 'Iota Inc'),
 ('Twisted Fate', 'Kappa Publishing'),
 ('Galactic War', 'Lambda Literature'),
 ('Whispering Woods', 'Mu Magazines'),
 ('Deserted Destinies', 'Nu Novels'),
 ('Cursed Kingdoms', 'Xi Xerox'),
 ('Hunted Houses', 'Omicron Oasis'),
 ('Timeless Tales', 'Pi Press'),
 ('Endless Embers', 'Rho Reads'),
 ('Frozen Fears', 'Sigma Stories')]

#- Assign an extra e-mail under the domain ###@alphapubs.com for each author publishing with 'Alpha Publications'. Adjust the database schema appropriately to achieve this.

In [ ]:
# cursor.execute(''' 
# SELECT authors.author_name, publishers.publisher_name, publishers.contact_email
# FROM authors
# JOIN publishers on authors.author_id = publishers.author_id
# ''')
# rows_ = cursor.fetchall()

# df_ = pd.DataFrame(rows_)
# # pd.read_sql("select * from books", conn)
# df


In [26]:
pd.read_sql_query("SELECT * from authors", conn)


,author_id,author_name,email
0,1,John Doe,john@example.com
1,2,Jane Smith,jane@example.com
2,3,Sarah Connor,sarah@example.com
3,4,Mike Tyson,mike@example.com
4,5,Lucy Loo,lucy@example.com
5,6,Brian Bryson,brian@example.com
6,7,Anna Smith,anna@example.com
7,8,Tim Track,tim@example.com
8,9,Liam Neeson,liam@example.com
9,10,Emily Em,emily@example.com


In [28]:
pd.read_sql_query("SELECT * from books", conn)

,book_id,author_id,publisher_id,title,genre,price
0,1,1,1,The Great Escape,Adventure,10.99
1,2,2,2,Mystery Lane,Mystery,12.99
2,3,1,1,Adventure in Space,Sci-fi,15.99
3,4,3,3,Ocean Blues,Romance,7.99
4,5,4,4,Lost in Dreams,Fantasy,9.99
5,6,5,5,Sunset Stories,Drama,8.99
6,7,6,6,Moonlit Secrets,Mystery,11.99
7,8,7,2,Starstruck,Romance,7.99
8,9,8,1,Dungeon Dive,Adventure,13.99
9,10,9,7,Hopes and Dreams,Drama,6.99


In [29]:
#chatgpt solution
# Step 1: Add a new column for the extra email in the authors table
try:
    cursor.execute("ALTER TABLE authors ADD COLUMN extra_email TEXT")
except sqlite3.OperationalError:
    print("Column 'extra_email' already exists.")  # Handle case if column already exists

# Step 2: Update the extra email for authors publishing with 'Alpha Publications'
# Assuming author_name is in "First Last" format

update_query = """
    UPDATE authors
    SET extra_email = LOWER(SUBSTR(author_name, 1, INSTR(author_name, ' ') - 1) || '.' || 
                            SUBSTR(author_name, INSTR(author_name, ' ') + 1) || 
                            '@alphapubs.com')
    WHERE author_id IN (
        SELECT author_id
        FROM books
        WHERE publisher_id = 1
    )
"""

# Execute the update query
cursor.execute(update_query)

# Commit changes
conn.commit()

# Step 3: Verify the update
select_query = """
    SELECT author_id, author_name, email, extra_email
    FROM authors
    WHERE extra_email IS NOT NULL
"""

# Fetch and display the results
cursor.execute(select_query)
authors_with_emails = cursor.fetchall()

for author in authors_with_emails:
    print(author)

# # Close the connection
# conn.close()




Column 'extra_email' already exists.
(1, 'John Doe', 'john@example.com', 'john.doe@alphapubs.com')
(8, 'Tim Track', 'tim@example.com', 'tim.track@alphapubs.com')


In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Select the columns to be one-hot encoded
columns_to_encode = ['job', 'education','marital','poutcome','age_category']  # Fix: Separate the column names into a list

# Initialize the OneHotEncoder
ohe_encoder = OneHotEncoder(handle_unknown='ignore')  # drop='first' avoids the dummy variable trap

# Apply the encoder to the specified columns
encoded_data = ohe_encoder.fit_transform(y_with_outcome[columns_to_encode])

# Convert the encoded data to a DataFrame
encoded_columns = ohe_encoder.get_feature_names_out(columns_to_encode)

# Convert the sparse matrix to a dense array
encoded_dense = encoded_data.toarray()

# Create a DataFrame from the dense array with appropriate column names
encoded_df = pd.DataFrame(encoded_dense, columns=encoded_columns)
encoded_df

# Reset index for both DataFrames
y_with_outcome = y_with_outcome.reset_index(drop=True)
encoded_df = encoded_df.reset_index(drop=True)

# Perform horizontal concatenation
horizontal_concat = pd.concat([y_with_outcome, encoded_df], axis=1)
ohe_df = horizontal_concat

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

ohe_df['housing'] = label_encoder.fit_transform(ohe_df['housing'])
print('HOUSING converted to number: ')
mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print(mapping)
print()
print('--------------------------------')

ohe_df['loan'] = label_encoder.fit_transform(ohe_df['loan'])
print('LOAN converted to number: ')
mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print(mapping)
print()
print('--------------------------------')

ohe_df['y'] = label_encoder.fit_transform(ohe_df['y'])
print('Y converted to number: ')
mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print(mapping)
print()
print('--------------------------------')

ohe_df = ohe_df.drop(['marital','job','education','marital','in_default','poutcome','age_category'], axis=1)

#transforming values into log scale to deal with outliers
ohe_df['duration'] = ohe_df['duration'].apply(
    lambda x: 0 if x <=0 else
              np.log(1+x)  )
print('DURATION made logaritmic (zero for balance <1): ')
print()
print('--------------------------------')

ohe_df['balance'] = ohe_df['balance'].apply(
    lambda x: 0 if x <=0 else
              np.log(1+x)  )
print('BALANCE made logaritmic (zero for balance <1): ')
print()
print('--------------------------------')

columns_transform = ['balance','duration','campaign','pdays','previous']
scaler = MinMaxScaler()
ohe_df[columns_transform] = scaler.fit_transform(ohe_df[columns_transform])
ohe_df

ohe_df = ohe_df.drop(['client_id'], axis =1)